**INSTALLATION**

In [ ]:
!git clone --recurse-submodules https://github.com/safednn-nasa/ProphecyPlus

fatal: destination path 'ProphecyPlus' already exists and is not an empty directory.


In [ ]:
%cd ./ProphecyPlus
!pwd
!ls -lt /content/ProphecyPlus/dataset_models/kj_tiny_taxinet/

/content/ProphecyPlus
/content/ProphecyPlus
total 108
-rw-r--r-- 1 root root  4902 Dec 30 16:22 KJ_TinyTaxiNet.onnx
-rw-r--r-- 1 root root     1 Dec 30 16:22 readme.txt
-rw-r--r-- 1 root root  5377 Dec 30 16:22 saved_model.pb
-rw-r--r-- 1 root root 45896 Dec 30 16:22 KJ_Taxinet.h5
-rw-r--r-- 1 root root 30274 Dec 30 16:22 KJ_TaxiNet.nnet
-rw-r--r-- 1 root root  5377 Dec 30 16:22 KJ_TaxiNet.pb


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/44

**LOAD DATA AND MODEL**

In [ ]:
import requests

# Dropbox shared link (ensure it's the correct URL)
shared_link = "https://www.dropbox.com/scl/fi/84wgpm5v7beta2jovs1wf/data_val.zip?rlkey=2dudlkuxqnailr6vysr2v2022&st=49oxo7dz&dl=0"

# Modify the link for direct download
direct_download_url = shared_link.replace("dl=0", "dl=1")

# Filepath to save the downloaded file
file_path = "data_val.zip"

try:
    # Send a GET request to download the file
    response = requests.get(direct_download_url)
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

    # Save the content to a local file
    with open(file_path, "wb") as file:
        file.write(response.content)
    print(f"File downloaded successfully and saved as {file_path}")

except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")

############################
!ls -lt ./data_val.zip

import zipfile
with zipfile.ZipFile('./data_val.zip', 'r') as zip_ref:
    zip_ref.extractall('.')


File downloaded successfully and saved as data_val.zip


In [ ]:
import glob
import PIL
import numpy as np
import pandas as pd
#import tensorflow as tf
#from tensorflow import keras

eval_folder = "./data_val/"
table = pd.read_csv(eval_folder + "errors.csv")

# Use each example image in folder
exampleImages = glob.glob(eval_folder + "*png")
imgNums = sorted([int( f.split("/")[-1].split(".")[0] ) for f in exampleImages])

x = list()
y = list()
for imgNum in imgNums:
  img = PIL.Image.open("{}{}.png".format(eval_folder,imgNum))
  img_copy = np.array(img)
  truth = np.array([table.CTE[imgNum],table.HE[imgNum]])
  img.close()

  #print(img_copy.shape)
  x.append(img_copy)
  y.append(truth)


x = np.array(x)
y = np.array(y)

print("Shape of X:", x.shape)

Shape of X: (7386, 200, 360, 3)


In [ ]:
def downsampleImage(img):
    """
    Function for downsampling images of taxiway from 200x360x3 into 8x16x1
    """

    img = np.array(img)

    # Remove yellow/orange lines
    mask = ((img[:,:,0].astype('float')-img[:,:,2].astype('float'))>60) & ((img[:,:,1].astype('float')-img[:,:,2].astype('float'))>30)
    img[mask] = 0

    # Convert to grayscale, crop out nose, sky, bottom of image, resize to 256x128, scale so
    # values range between 0 and 1
    img = np.array(PIL.Image.fromarray(img).convert('L').crop((55,5,360,140)).resize((256,128)))/255.0

    # Downsample image
    # Split image into stride x stride boxes, average numPix brightest pixels in that box
    # As a result, img2 has one value for every box
    img2 = np.zeros((height,width))
    for i in range(height):
        for j in range(width):
            img2[i,j] = np.mean(np.sort(img[stride*i:stride*(i+1),stride*j:stride*(j+1)].reshape(-1))[-numPix:])

    # Ensure that the mean of the image is 0.5 and that values range between 0 and 1
    # The training data only contains images from sunny, 9am conditions.
    # Biasing the image helps the network generalize to different lighting conditions (cloudy, noon, etc)
    img2 -= img2.mean()
    img2 += 0.5
    img2[img2>1] = 1
    img2[img2<0] = 0
    return img2

In [ ]:
import tensorflow as tf
import keras

model = keras.models.load_model("/content/ProphecyPlus/dataset_models/kj_tiny_taxinet/KJ_Taxinet.h5")
model.summary()

for layer in model.layers:
  print("LAYER NAME:", layer.name)

#model.eval()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/lambda_layer.py:327: UserWarning: onnx2keras.utils is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/lambda_layer.py:327: UserWarning: onnx2keras.elementwise_layers is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


Tensor("Placeholder:0", shape=(None, 1, 2, 8), dtype=float32) Tensor("Placeholder_1:0", shape=(8,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 8), dtype=float32) Tensor("Placeholder_1:0", shape=(8,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 8), dtype=float32) Tensor("Placeholder_1:0", shape=(8,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 2), dtype=float32) Tensor("Placeholder_1:0", shape=(2,), dtype=float32)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_0 (InputLayer)        [(None, 8, 16, 1)]           0         []                            
                                                                                                  
 layer_0_ (Permute)          (None, 1, 8, 16)             0         ['input_0[0][0]']             
                                              

In [ ]:
### IMPORTANT PARAMETERS FOR IMAGE PROCESSING ###
stride = 16             # Size of square of pixels downsampled to one grayscale value
numPix = 16             # During downsampling, average the numPix brightest pixels in each square
width  = 256//stride    # Width of downsampled grayscale image
height = 128//stride    # Height of downsampled grayscale image
#################################################

print("height:", height, ",width:", width)

x_train = x[0:int(0.80*len(x))]
y_train = y[0:int(0.80*len(y))]

print("TRAIN:",np.shape(x_train), np.shape(y_train))

x_test = x[int(0.80*len(x)):]
y_test = y[int(0.80*len(y)):]

print("TEST:",np.shape(x_test), np.shape(y_test))

error_train_ct = []
error_train_he = []

pred_train_cte = []
pred_train_he = []


flat_img_keras_lst = []

for imgNum, img in enumerate(x_train):
  dsImg = downsampleImage(img)
  # Compute prediction and ground truth of CTE / HE
  #print("Img Shape:", img.shape)
  flat_img = dsImg.reshape(-1)
  #print("Flat Img Shape:", flat_img.shape)

  flat_img_keras = flat_img.reshape(1, height, width, 1)
  flat_img_keras_lst.append(flat_img.reshape(height, width, 1))
  #print("Flat Img Keras Shape:", flat_img_keras.shape)



  predictions = model.predict(flat_img_keras)
  #print("Prediction Keras:", predictions)
  truth = y_train[imgNum]
  #print("Truth:", truth)

  error_train_ct.append(predictions[0][0]-truth[0])
  error_train_he.append(predictions[0][1]-truth[1])

  pred_train_cte.append(np.abs(predictions[0][0]))
  pred_train_he.append(np.abs(predictions[0][1]))

  # print("\nImg Number: %d" % imgNum)
 # print("Prediction Keras: %.3f CTE, %.3f HE" % (predictions[0][0],predictions[0][1]))
 # print("Truth: %.3f CTE, %.3f HE" % (truth[0],truth[1]))


np.save('./x_train_npy.npy', np.array(flat_img_keras_lst))

################################################################


error_test_ct = []
error_test_he = []

pred_test_cte = []
pred_test_he = []

flat_img_keras_lst = []

for imgNum, img in enumerate(x_test):
  dsImg = downsampleImage(img)
  # Compute prediction and ground truth of CTE / HE
  #print("Img Shape:", img.shape)
  flat_img = dsImg.reshape(-1)
  #print("Flat Img Shape:", flat_img.shape)

  flat_img_keras = flat_img.reshape(1, height, width, 1)
  #print("Flat Img Keras Shape:", flat_img_keras.shape)
  flat_img_keras_lst.append(flat_img.reshape(height, width, 1))

  predictions = model.predict(flat_img_keras)
  #print("Prediction Keras:", predictions)
  truth = y_test[imgNum]
  #print("Truth:", truth)

  error_test_ct.append(predictions[0][0]-truth[0])
  error_test_he.append(predictions[0][1]-truth[1])

  pred_test_cte.append(np.abs(predictions[0][0]))
  pred_test_he.append(np.abs(predictions[0][1]))

 # print("\nImg Number: %d" % imgNum)
 # print("Prediction Keras: %.3f CTE, %.3f HE" % (predictions[0][0],predictions[0][1]))
 # print("Truth: %.3f CTE, %.3f HE" % (truth[0],truth[1]))
np.save('./x_test_npy.npy', np.array(flat_img_keras_lst))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 20ms/step


**EVALUATE ACCURACY IN TERMS OF MAE ON TRAIN AND TEST DATA**

In [ ]:
print(" TRAIN ")
print("Cross Track Error")
abs_error_train=np.abs(error_train_ct)
print('MeanAbsoluteError: '+str(np.mean(abs_error_train)))
print('MedianAbsoluteError: '+str(np.median(abs_error_train)))
print('StandardDeviation: '+str(np.std(abs_error_train)))
print('MaxAbsoluteError: '+str(np.max(abs_error_train)))

print(" TRAIN ")
print("Heading Error")
abs_error_train=np.abs(error_train_he)
print('MeanAbsoluteError: '+str(np.mean(abs_error_train)))
print('MedianAbsoluteError: '+str(np.median(abs_error_train)))
print('StandardDeviation: '+str(np.std(abs_error_train)))
print('MaxAbsoluteError: '+str(np.max(abs_error_train)))

print(" TEST ")
print("Cross Track Error")
abs_error_test=np.abs(error_test_ct)
print('MeanAbsoluteError: '+str(np.mean(abs_error_test)))
print('MedianAbsoluteError: '+str(np.median(abs_error_test)))
print('StandardDeviation: '+str(np.std(abs_error_test)))
print('MaxAbsoluteError: '+str(np.max(abs_error_test)))

print(" TEST ")
print("Heading Error")
abs_error_test=np.abs(error_test_he)
print('MeanAbsoluteError: '+str(np.mean(abs_error_test)))
print('MedianAbsoluteError: '+str(np.median(abs_error_test)))
print('StandardDeviation: '+str(np.std(abs_error_test)))
print('MaxAbsoluteError: '+str(np.max(abs_error_test)))

 TRAIN 
Cross Track Error
MeanAbsoluteError: 1.3419898688496428
MedianAbsoluteError: 1.1240889876785276
StandardDeviation: 1.0314191135748914
MaxAbsoluteError: 9.604623790901185
 TRAIN 
Heading Error
MeanAbsoluteError: 2.790885690451818
MedianAbsoluteError: 2.3334284747924805
StandardDeviation: 2.397462409773755
MaxAbsoluteError: 24.663099521484376
 TEST 
Cross Track Error
MeanAbsoluteError: 1.811774308290391
MedianAbsoluteError: 1.9239859447174066
StandardDeviation: 1.1206498607945428
MaxAbsoluteError: 5.817006080551147
 TEST 
Heading Error
MeanAbsoluteError: 2.596350178839501
MedianAbsoluteError: 2.1431100562896725
StandardDeviation: 2.1977444523675893
MaxAbsoluteError: 15.552720513183594


**CALCULATE MEAN ABSOLUTE MODEL OUTPUT VALUES FOR CTE AND HE.**

**The MEAN ABS output value on train data is taken as the safety threshold.**

In [ ]:
print("AVERAGE TRAIN CTE:",np.mean(pred_train_cte))
print("AVERAGE TRAIN HE:",np.mean(pred_train_he))
print("AVERAGE TEST CTE:",np.mean(pred_test_cte))
print("AVERAGE TEST HE:",np.mean(pred_test_he))

AVERAGE TRAIN CTE: 4.276557
AVERAGE TRAIN HE: 8.026427
AVERAGE TEST CTE: 5.2611675
AVERAGE TEST HE: 7.742248


In [ ]:
labels_train_cte = []
labels_test_cte = []
labels_train_he = []
labels_test_he = []

for i in range(len(pred_train_cte)):
  if (pred_train_cte[i] > 4.276557):
    labels_train_cte.append(0)
  else:
    labels_train_cte.append(1)

for i in range(len(pred_test_cte)):
  if (pred_test_cte[i] > 4.276557):
    labels_test_cte.append(0)
  else:
    labels_test_cte.append(1)


for i in range(len(pred_train_he)):
  if (pred_train_he[i] > 8.026427):
    labels_train_he.append(0)
  else:
    labels_train_he.append(1)

for i in range(len(pred_test_he)):
  if (pred_test_he[i] > 8.026427):
    labels_test_he.append(0)
  else:
    labels_test_he.append(1)


print("TRAIN CTE 0:", labels_train_cte.count(0))
print("TRAIN CTE 1:", labels_train_cte.count(1))
print("TEST CTE 0:", labels_test_cte.count(0))
print("TEST CTE 1:", labels_test_cte.count(1))

print("TRAIN HE 0:", labels_train_he.count(0))
print("TRAIN HE 1:", labels_train_he.count(1))
print("TEST HE 0:", labels_test_he.count(0))
print("TEST HE 1:", labels_test_he.count(1))


np.save('./y_train_cte_npy.npy', np.array(labels_train_cte))
np.save('./y_test_cte_npy.npy', np.array(labels_test_cte))

np.save('./y_train_he_npy.npy', np.array(labels_train_he))
np.save('./y_test_he_npy.npy', np.array(labels_test_he))

TRAIN CTE 0: 2995
TRAIN CTE 1: 2913
TEST CTE 0: 1143
TEST CTE 1: 335
TRAIN HE 0: 2905
TRAIN HE 1: 3003
TEST HE 0: 684
TEST HE 1: 794


In [ ]:
x_train = np.load('./x_train_npy.npy')
x_test = np.load('./x_test_npy.npy')

print(np.shape(x_train))
print(np.shape(x_test))


(5908, 8, 16, 1)
(1478, 8, 16, 1)


**INVOKE PROPHECY TO EXTRACT RULES**

**MODEL NEEDS TO BE EXECUTED GPU**

In [ ]:
import sys
import argparse
import numpy as np
import pandas as pd

from pathlib import Path

from prophecy.utils.misc import get_model, read_split
from prophecy.core.extract import Extractor
from prophecy.core.detect import RulesDetector, ClassifierDetector
from prophecy.utils.paths import results_path


def run_analyze_command(model,train_features,train_labels,val_features,val_labels,l_name,typ,inptype,acts,top,working_dir):
    train_features, train_labels = read_split(train_features, train_labels)
    val_features, val_labels = read_split(val_features, val_labels)

    rule_extractor = Extractor(model=model, train_features=train_features, train_labels=train_labels,
                               val_features=val_features, val_labels=val_labels, skip_rules=False, layer_name=l_name,
                               only_dense=False, balance=False, confidence=False,
                               only_activation=False, type=typ, inptype=inptype, acts=acts, top=top)

    classifiers_path = working_dir / "classifiers"
    ruleset = rule_extractor(path=classifiers_path)
    rules_path = working_dir / 'ruleset.csv'
    pd.DataFrame(ruleset).to_csv(rules_path, index=False)


**PARAMETERS: rules per label (based on given labels array), rules in terms of on/off activation values, TOP rules (highest train recall), all dense and activation layers.**

In [ ]:
import os

directory_path = '/content/ProphecyPlus/results/kjt/rules_cte_1/'
os.makedirs(directory_path,exist_ok=True)

wd = Path('/content/ProphecyPlus/results/kjt/rules_cte_1/')
run_analyze_command(model,"./x_train_npy.npy","./y_train_cte_npy.npy","./x_test_npy.npy","./y_test_cte_npy.npy","layer_10",3,0,False,False,wd)

CONFIG PARAMS: LAYER NAME: layer_10 ,TYPE: 3 ,INP TYPE: 0 ,ACTS: False ,Top/All: False
Layer Name: layer_10
Layers to be considered for fingerprinting: ['layer_10']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_10 layer
Processing layer_10:   0%|          | 0/24 [00:00<?, ?it/s]tf.Tensor(
[[[[-0.26016584 -0.43339378  2.2210205  ...  2.5751863   4.813346
    -0.2545323 ]
   [ 0.98201674 -0.43722454  3.0293176  ...  4.171374    4.887096
    -0.29122013]]]


 [[[-0.18652447 -0.4324616   2.2048576  ...  2.6458187   4.772342
    -0.25256512]
   [ 1.0005517  -0.4410913   2.9933293  ...  4.2622576   4.9028606
    -0.29046923]]]


 [[[-0.11948881 -0.43224463  2.210664   ...  2.6556752   4.703244
    -0.24672768]
   [ 1.0262357  -0.44452363  3.0207863  ...  4.361699    4.8975844
    -0.2941592 ]]]


 ...


 [[[ 0.9637208  -0.42356405  2.7032819  ...  2.80697     4.7505593
    -0.23325606]
   [ 1.5171398  -0.45405763  3.1704452  ...  3.8166356   6.071677
 

In [ ]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/kjt/rules_cte_1/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]


df_op.to_csv("./rules_cte_10_1.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_cte_10_0.csv",index=False)

#df_op = df_op.reset_index()  # Reset index to make 'index' column
#df_op
#top_df_cte_13 = df_op[df_op['index'] == 0]  # Use 'index' column instead of 'row'

#top_df_cte_13

****** RULES ********


In [ ]:
wd = Path('/content/ProphecyPlus/results/kjt/rules_cte_1/')
run_analyze_command(model,"./x_train_npy.npy","./y_train_he_npy.npy","./x_test_npy.npy","./y_test_he_npy.npy","layer_10",3,0,False,False,wd)

CONFIG PARAMS: LAYER NAME: layer_10 ,TYPE: 3 ,INP TYPE: 0 ,ACTS: False ,Top/All: False
Layer Name: layer_10
Layers to be considered for fingerprinting: ['layer_10']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_10 layer
Processing layer_10:   0%|          | 0/24 [00:00<?, ?it/s]tf.Tensor(
[[[[-0.26016584 -0.43339378  2.2210205  ...  2.5751863   4.813346
    -0.2545323 ]
   [ 0.98201674 -0.43722454  3.0293176  ...  4.171374    4.887096
    -0.29122013]]]


 [[[-0.18652447 -0.4324616   2.2048576  ...  2.6458187   4.772342
    -0.25256512]
   [ 1.0005517  -0.4410913   2.9933293  ...  4.2622576   4.9028606
    -0.29046923]]]


 [[[-0.11948881 -0.43224463  2.210664   ...  2.6556752   4.703244
    -0.24672768]
   [ 1.0262357  -0.44452363  3.0207863  ...  4.361699    4.8975844
    -0.2941592 ]]]


 ...


 [[[ 0.9637208  -0.42356405  2.7032819  ...  2.80697     4.7505593
    -0.23325606]
   [ 1.5171398  -0.45405763  3.1704452  ...  3.8166356   6.071677
 

In [ ]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/kjt/rules_cte_1/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]


df_op.to_csv("./rules_he_10_1.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_he_10_0.csv",index=False)

#df_op = df_op.reset_index()  # Reset index to make 'index' column
#df_op
#top_df_cte_13 = df_op[df_op['index'] == 0]  # Use 'index' column instead of 'row'

#top_df_cte_13

****** RULES ********


In [ ]:
print("****** RULES ********")


df_op = pd.read_csv("./rules_cte_10_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("CTE 10 1:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_he_10_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("HE 10 1:", train_f1,",",test_f1)



#print("****** CTE 1 RULES ********")
df_op = pd.read_csv("./rules_cte_10_1.csv")
df_op = df_op[df_op.index == 0]
layer_10_cte_rule_neurons = df_op['neurons']
layer_10_cte_rule_signature = df_op['signature']


#print("****** HE 1 RULES ********")
df_op = pd.read_csv("./rules_he_10_1.csv")
df_op = df_op[df_op.index == 0]
layer_10_he_rule_neurons = df_op['neurons']
layer_10_he_rule_signature = df_op['signature']

print(layer_10_cte_rule_neurons.array[0])
print(layer_10_cte_rule_signature.array[0])
print(layer_10_he_rule_neurons.array[0])
print(layer_10_he_rule_signature.array[0])


****** RULES ********
CTE 10 1: 21.88933047997554 , 1.1869436201780417
HE 10 1: 57.44125326370757 , 25.46652030735456
[5, 6, 7, 5, 5, 7, 2, 2]
['<=', 5.014780044555664, '<=', 8.457788467407227, '<=', 6.801446199417114, '<=', 4.407367944717407, '<=', 0.42133453488349915, '<=', 4.016632080078125, '<=', 0.710400253534317, '<=', -0.7180337011814117]
[2, 7, 4, 7, 3, 7, 6, 1, 1, 7, 7, 4, 2, 7, 7]
['<=', 5.0200934410095215, '<=', 5.228757619857788, '<=', 0.7042066156864166, '>', 2.3984490633010864, '>', -4.647291898727417, '>', 2.8580180406570435, '>', 0.23533541709184647, '<=', -0.8676009178161621, '>', -1.7780373096466064, '<=', 5.001114130020142, '>', 3.032026529312134, '<=', 0.6818224787712097, '<=', 4.339860677719116, '>', 3.0972129106521606, '<=', 4.9778783321380615]


In [ ]:
print("****** RULES ********")


df_op = pd.read_csv("/content/ProphecyPlus/rules_cte_10_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("CTE 10 0:", train_f1,",",test_f1)



df_op = pd.read_csv("/content/ProphecyPlus/rules_he_10_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("HE 10 0:", train_f1,",",test_f1)



#print("****** CTE 1 RULES ********")
df_op = pd.read_csv("/content/ProphecyPlus/rules_cte_10_0.csv")
df_op = df_op[df_op.index == 0]
layer_10_cte_rule_neurons0 = df_op['neurons']
layer_10_cte_rule_signature0 = df_op['signature']


#print("****** HE 1 RULES ********")
df_op = pd.read_csv("/content/ProphecyPlus/rules_he_10_0.csv")
df_op = df_op[df_op.index == 0]
layer_10_he_rule_neurons0 = df_op['neurons']
layer_10_he_rule_signature0 = df_op['signature']

print(layer_10_cte_rule_neurons0.array[0])
print(layer_10_cte_rule_signature0.array[0])
print(layer_10_he_rule_neurons0.array[0])
print(layer_10_he_rule_signature0.array[0])


****** RULES ********
CTE 10 0: 25.007303534910896 , 68.69615163699024
HE 10 0: 15.027061445399555 , 15.384615384615383
[5, 6, 5, 2, 7, 5, 7, 6]
['<=', 5.014780044555664, '>', 8.457788467407227, '<=', 0.45192472636699677, '>', -0.7284570932388306, '>', 3.306133508682251, '<=', 0.23589569330215454, '>', 3.5442665815353394, '>', 8.710090160369873]
[2, 7, 4, 7, 1, 1, 5]
['<=', 5.0200934410095215, '<=', 5.228757619857788, '>', 0.7042066156864166, '<=', 3.8628121614456177, '<=', -1.4256365299224854, '<=', -1.727737545967102, '>', -3.598828911781311]


**Prophecy helper methods**

In [ ]:
def check_pattern(layer_vals: list, neuron_ids: list, neuron_sig: list) -> bool:
    """
        Check if the provided layer values satisfy the provided neuron signature.
    :param layer_vals:
    :param neuron_ids:
    :param neuron_sig:
    :return:
    """
    found = True
    oper = -1
    # layer_vals = (layer_vals).flatten()

    for ind in range(0, len(neuron_sig)):
        if ind % 2 == 0:
            op = neuron_sig[ind]
            if op == '<=':
                oper = 0
            else:
                oper = 1
        else:
            v = int(neuron_ids[(int)(ind / 2)])
            vsig = float(neuron_sig[ind])
            val = float(layer_vals[v])
            # print(v,vsig,val,oper)
            if oper == 0:
                if val > vsig:
                    # print(v,val,vsig,oper)
                    found = False
                    break
            else:
                if val <= vsig:
                    # print(v,val,vsig,oper)
                    found = False
                    break
            oper = -1

    return found

def get_suffix_cluster(neuron_ids, neuron_sig, suffixes, VAL=False):
    # Get the cluster of inputs that such that all inputs in the cluster
    # have provided on/off signature for the provided neurons.
    #
    # The returned cluster is an array of indices (into mnist.train.images).
    if (VAL == False):
        return np.where((suffixes[:, neuron_ids] == neuron_sig).all(axis=1))[0]

    matched_ids = []
    # print(len(suffixes))
    for indx in range(0, len(suffixes)):
        if (check_pattern(suffixes[indx], neuron_ids, neuron_sig) == True):
            matched_ids.append(indx)
    # print(matched_ids)
    return matched_ids



**Get fingerprints (neuron values) after layer_10 on train data**

In [ ]:
from keras import backend

func = None


for layer in model.layers:
    print(layer.name)
    if (layer.name == 'layer_10'):
      func = backend.function(model.input, [layer.output])


fingerprint_10 = []

if (func != None):
  fingerprint_10 = func(x_train)

input_0
layer_0_
layer_1_
layer_2_
layer_3__const2
layer_3_
layer_4_
layer_6_
layer_7_
layer_8__const2
layer_8_
layer_9_
layer_10
layer_11_const2
layer_11
layer_12
layer_13
layer_14_const2
layer_14
tf.Tensor(
[[[[-0.26016584 -0.43339378  2.2210205  ...  2.5751863   4.813346
    -0.2545323 ]
   [ 0.98201674 -0.43722454  3.0293176  ...  4.171374    4.887096
    -0.29122013]]]


 [[[-0.18652447 -0.4324616   2.2048576  ...  2.6458187   4.772342
    -0.25256512]
   [ 1.0005517  -0.4410913   2.9933293  ...  4.2622576   4.9028606
    -0.29046923]]]


 [[[-0.11948881 -0.43224463  2.210664   ...  2.6556752   4.703244
    -0.24672768]
   [ 1.0262357  -0.44452363  3.0207863  ...  4.361699    4.8975844
    -0.2941592 ]]]


 ...


 [[[ 1.9117573  -0.47347075  2.9767928  ...  3.8442156   6.0059633
    -0.35950464]
   [ 0.7352615  -0.43866083  2.328126   ...  3.0062077   4.824507
    -0.26145965]]]


 [[[ 1.97178    -0.47032604  3.008598   ...  3.872708    5.998828
    -0.35775897]
   [ 0.70467544 -0

In [ ]:
print(len(x_train))
x_train_flat = []
for indx in range(0, len(x_train)):
  x_train_flat.append((x_train[indx]).flatten())

x_train_flat = np.array(x_train_flat)
print(np.shape(x_train))
print(np.shape(x_train_flat))

print(np.shape(fingerprint_10[0]))

5908
(5908, 8, 16, 1)
(5908, 128)
(5908, 8)


**Get the values to set the lower and upper bounds for the input vars and inner layer variables, based on the rules.**

**For each rule, the longest sequence of consecutive images satisfying the rule are considered.**

In [ ]:
length = len(x_train_flat[0])


x_train_min = np.zeros(length)
x_train_max = np.zeros(length)

for indx in range(0,length):
  x_train_min[indx] = np.min(x_train_flat[:,indx])
  x_train_max[indx] = np.max(x_train_flat[:,indx])

print(x_train_min)
print(x_train_max)


print("CTE RULE:")
###########
rule_neurons_list_cte = []
rule_neurons = (layer_10_cte_rule_neurons.array[0]).split(",")
for indx in range(0, len(rule_neurons)):
    rule_neurons[indx] = (rule_neurons[indx]).strip()
    rule_neurons[indx] = (rule_neurons[indx]).replace("[", "")
    rule_neurons[indx] = (rule_neurons[indx]).replace("]","")
    rule_neurons_list_cte.append(int(rule_neurons[indx]))

print(rule_neurons_list_cte)

rule_sig_list_cte = []
rule_sig = (layer_10_cte_rule_signature.array[0]).split(",")
for indx in range(0, len(rule_sig)):
    rule_sig[indx] = (rule_sig[indx]).strip()
    rule_sig[indx] = (rule_sig[indx]).replace("[", "")
    rule_sig[indx] = (rule_sig[indx]).replace("]","")
    if (indx % 2 == 0):
      rule_sig[indx] = (rule_sig[indx]).replace("'", "")
      rule_sig_list_cte.append(rule_sig[indx])
    else:
      rule_sig_list_cte.append(float(rule_sig[indx]))

print(rule_sig_list_cte)

fngprnt = fingerprint_10[0]
indices = get_suffix_cluster(rule_neurons_list_cte, rule_sig_list_cte, fngprnt,VAL=True)
print("indices:", len(indices))

print("SORTED:")
sorted_indices = sorted(indices)
print(sorted_indices)
consecutive_min= [sorted_indices[0]]
consecutive_m = 0
consecutive_max = []
consecutive_l = 0
consecutive_len = []
for indx in range(1, len(sorted_indices)):
  if (sorted_indices[indx] - sorted_indices[indx-1] == 1):
    consecutive_m = sorted_indices[indx]
    consecutive_l = consecutive_l + 1
  else:
    consecutive_max.append(sorted_indices[indx-1])
    consecutive_min.append(sorted_indices[indx])
    consecutive_len.append(consecutive_l)
    consecutive_l = 0
    consecutive_m = 0

print("MIN:", consecutive_min)
print("MAX:", consecutive_max)
print("LEN:",np.max(consecutive_len), consecutive_len.index(np.max(consecutive_len)))



x_train_cte = []
fngprnt_cte = []
inp_ex = []
finger_ex = []
#for indx in range(0, len(indices)):
#    if (indx == 0):
for indx in range(consecutive_min[0], consecutive_max[0]):
    if (indx == consecutive_min[0]):
      inp_ex = x_train_flat[indx]
      finger_ex = fingerprint_10[0][indx]

    x_train_cte.append(x_train_flat[indx])
    fngprnt_cte.append(fingerprint_10[0][indx])

x_train_cte = np.array(x_train_cte)
fngprnt_cte = np.array(fngprnt_cte)

print(np.shape(x_train_cte))
x_train_minCTE = np.zeros(length)
x_train_maxCTE = np.zeros(length)

for indx in range(0,length):
  x_train_minCTE[indx] = np.min(x_train_cte[:,indx])
  x_train_maxCTE[indx] = np.max(x_train_cte[:,indx])

print(x_train_minCTE)
print(x_train_maxCTE)

print(np.shape(fngprnt_cte))
fngprnt_min_cte = np.zeros(len(fngprnt_cte[0]))
fngprnt_max_cte = np.zeros(len(fngprnt_cte[0]))

for indx in range(0,len(fngprnt_cte[0])):
  fngprnt_min_cte[indx] = np.min(fngprnt_cte[:,indx])
  fngprnt_max_cte[indx] = np.max(fngprnt_cte[:,indx])

print(fngprnt_min_cte)
print(fngprnt_max_cte)


print("INP:", inp_ex)
print("FINGER:", finger_ex)

[0.36404144 0.36037263 0.32961474 0.32884881 0.35045381 0.35795037
 0.37650888 0.39604205 0.39824793 0.39785348 0.39858877 0.3939185
 0.38999694 0.3894244  0.39050054 0.38926164 0.22116843 0.18043045
 0.17305645 0.20998392 0.2158835  0.27976984 0.33289101 0.30152229
 0.2659735  0.27905178 0.28924632 0.36837278 0.39922832 0.3946672
 0.35002872 0.38987247 0.14694968 0.19119753 0.22777076 0.25891353
 0.21280254 0.26782514 0.29537377 0.29710095 0.27209903 0.23497817
 0.21542777 0.23297335 0.22390472 0.30461282 0.32118758 0.35895374
 0.18875804 0.23876379 0.23973269 0.20781633 0.25474686 0.2602271
 0.27331495 0.2810566  0.29480124 0.26616498 0.23485562 0.21879404
 0.21448185 0.22192287 0.21875766 0.25191674 0.2076957  0.22034888
 0.20951095 0.18575751 0.25643574 0.25178271 0.25221163 0.27322495
 0.28897059 0.28653301 0.26131855 0.23656173 0.22149778 0.22565679
 0.19116307 0.21677581 0.20153761 0.20362094 0.17176585 0.19518229
 0.24211857 0.24609949 0.2096718  0.2654316  0.27264476 0.2599992

In [ ]:
print("HE RULE:")
###########
rule_neurons_list_he = []
rule_neurons = (layer_10_he_rule_neurons.array[0]).split(",")
for indx in range(0, len(rule_neurons)):
    rule_neurons[indx] = (rule_neurons[indx]).strip()
    rule_neurons[indx] = (rule_neurons[indx]).replace("[", "")
    rule_neurons[indx] = (rule_neurons[indx]).replace("]","")
    rule_neurons_list_he.append(int(rule_neurons[indx]))

print(rule_neurons_list_he)

rule_sig_list_he = []
rule_sig = (layer_10_he_rule_signature.array[0]).split(",")
for indx in range(0, len(rule_sig)):
    rule_sig[indx] = (rule_sig[indx]).strip()
    rule_sig[indx] = (rule_sig[indx]).replace("[", "")
    rule_sig[indx] = (rule_sig[indx]).replace("]","")
    if (indx % 2 == 0):
      rule_sig[indx] = (rule_sig[indx]).replace("'", "")
      rule_sig_list_he.append(rule_sig[indx])
    else:
      rule_sig_list_he.append(float(rule_sig[indx]))

print(rule_sig_list_he)

fngprnt = fingerprint_10[0]
indices = get_suffix_cluster(rule_neurons_list_he, rule_sig_list_he, fngprnt,VAL=True)
print("indices:", len(indices))

print("SORTED:")
sorted_indices = sorted(indices)
print(sorted_indices)
consecutive_min= [sorted_indices[0]]
consecutive_m = 0
consecutive_max = []
consecutive_l = 0
consecutive_len = []
for indx in range(1, len(sorted_indices)):
  if (sorted_indices[indx] - sorted_indices[indx-1] == 1):
    consecutive_m = sorted_indices[indx]
    consecutive_l = consecutive_l + 1
  else:
    consecutive_max.append(sorted_indices[indx-1])
    consecutive_min.append(sorted_indices[indx])
    consecutive_len.append(consecutive_l)
    consecutive_l = 0
    consecutive_m = 0

print("MIN:", consecutive_min)
print("MAX:", consecutive_max)

max_indx = consecutive_len.index(np.max(consecutive_len))
print("LEN:",np.max(consecutive_len), max_indx)



x_train_he = []
fngprnt_he = []
inp_he_ex = []
finger_he_ex = []

for indx in range(consecutive_min[max_indx], consecutive_max[max_indx]):
    if (indx == consecutive_min[max_indx]):
      inp_he_ex = x_train_flat[indx]
      finger_he_ex = fingerprint_10[0][indx]

    x_train_he.append(x_train_flat[indx])
    fngprnt_he.append(fingerprint_10[0][indx])

x_train_he = np.array(x_train_he)
fngprnt_he = np.array(fngprnt_he)

print(np.shape(x_train_he))
x_train_minhe = np.zeros(length)
x_train_maxhe = np.zeros(length)

for indx in range(0,length):
  x_train_minhe[indx] = np.min(x_train_he[:,indx])
  x_train_maxhe[indx] = np.max(x_train_he[:,indx])

print(x_train_minhe)
print(x_train_maxhe)

print(np.shape(fngprnt_he))
fngprnt_min_he = np.zeros(len(fngprnt_he[0]))
fngprnt_max_he = np.zeros(len(fngprnt_he[0]))

for indx in range(0,len(fngprnt_he[0])):
  fngprnt_min_he[indx] = np.min(fngprnt_he[:,indx])
  fngprnt_max_he[indx] = np.max(fngprnt_he[:,indx])

print(fngprnt_min_he)
print(fngprnt_max_he)


print("INP:", inp_he_ex)
print("FINGER:", finger_he_ex)

HE RULE:
[2, 7, 4, 7, 3, 7, 6, 1, 1, 7, 7, 4, 2, 7, 7]
['<=', 5.0200934410095215, '<=', 5.228757619857788, '<=', 0.7042066156864166, '>', 2.3984490633010864, '>', -4.647291898727417, '>', 2.8580180406570435, '>', 0.23533541709184647, '<=', -0.8676009178161621, '>', -1.7780373096466064, '<=', 5.001114130020142, '>', 3.032026529312134, '<=', 0.6818224787712097, '<=', 4.339860677719116, '>', 3.0972129106521606, '<=', 4.9778783321380615]
indices: 1210
SORTED:
[161, 162, 163, 164, 165, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 214, 215, 225, 234, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 327, 357, 365, 370, 371, 381, 382, 383, 386, 387, 388, 389, 390, 391, 392, 395, 396, 397, 442, 443, 444, 445, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 495, 500, 501, 509, 510, 511, 513, 514, 515, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 564, 565,

In [ ]:
print("CTE 0 RULE:")
###########
rule_neurons_list_cte0 = []
rule_neurons = (layer_10_cte_rule_neurons0.array[0]).split(",")
for indx in range(0, len(rule_neurons)):
    rule_neurons[indx] = (rule_neurons[indx]).strip()
    rule_neurons[indx] = (rule_neurons[indx]).replace("[", "")
    rule_neurons[indx] = (rule_neurons[indx]).replace("]","")
    rule_neurons_list_cte0.append(int(rule_neurons[indx]))

print(rule_neurons_list_cte0)

rule_sig_list_cte0 = []
rule_sig = (layer_10_cte_rule_signature0.array[0]).split(",")
for indx in range(0, len(rule_sig)):
    rule_sig[indx] = (rule_sig[indx]).strip()
    rule_sig[indx] = (rule_sig[indx]).replace("[", "")
    rule_sig[indx] = (rule_sig[indx]).replace("]","")
    if (indx % 2 == 0):
      rule_sig[indx] = (rule_sig[indx]).replace("'", "")
      rule_sig_list_cte0.append(rule_sig[indx])
    else:
      rule_sig_list_cte0.append(float(rule_sig[indx]))

print(rule_sig_list_cte0)

fngprnt = fingerprint_10[0]
indices = get_suffix_cluster(rule_neurons_list_cte0, rule_sig_list_cte0, fngprnt,VAL=True)
print("indices:", len(indices))

print("SORTED:")
sorted_indices = sorted(indices)
print(sorted_indices)
consecutive_min= [sorted_indices[0]]
consecutive_m = 0
consecutive_max = []
consecutive_l = 0
consecutive_len = []
for indx in range(1, len(sorted_indices)):
  if (sorted_indices[indx] - sorted_indices[indx-1] == 1):
    consecutive_m = sorted_indices[indx]
    consecutive_l = consecutive_l + 1
  else:
    consecutive_max.append(sorted_indices[indx-1])
    consecutive_min.append(sorted_indices[indx])
    consecutive_len.append(consecutive_l)
    consecutive_l = 0
    consecutive_m = 0

print("MIN:", consecutive_min)
print("MAX:", consecutive_max)
max_indx = consecutive_len.index(np.max(consecutive_len))
print("LEN:",np.max(consecutive_len), max_indx)



x_train_cte0 = []
fngprnt_cte0 = []
inp_ex0 = []
finger_ex0 = []
for indx in range(consecutive_min[max_indx], consecutive_max[max_indx]):
    if (indx == consecutive_min[max_indx]):
      inp_ex0 = x_train_flat[indx]
      finger_ex0 = fingerprint_10[0][indx]

    x_train_cte0.append(x_train_flat[indx])
    fngprnt_cte0.append(fingerprint_10[0][indx])

x_train_cte0 = np.array(x_train_cte0)
fngprnt_cte0 = np.array(fngprnt_cte0)

print(np.shape(x_train_cte0))
x_train_minCTE0 = np.zeros(length)
x_train_maxCTE0 = np.zeros(length)

for indx in range(0,length):
  x_train_minCTE0[indx] = np.min(x_train_cte0[:,indx])
  x_train_maxCTE0[indx] = np.max(x_train_cte0[:,indx])

print(x_train_minCTE0)
print(x_train_maxCTE0)

print(np.shape(fngprnt_cte0))
fngprnt_min_cte0 = np.zeros(len(fngprnt_cte0[0]))
fngprnt_max_cte0 = np.zeros(len(fngprnt_cte0[0]))

for indx in range(0,len(fngprnt_cte0[0])):
  fngprnt_min_cte0[indx] = np.min(fngprnt_cte0[:,indx])
  fngprnt_max_cte0[indx] = np.max(fngprnt_cte0[:,indx])

print(fngprnt_min_cte0)
print(fngprnt_max_cte0)


print("INP:", inp_ex0)
print("FINGER:", finger_ex0)

CTE 0 RULE:
[5, 6, 5, 2, 7, 5, 7, 6]
['<=', 5.014780044555664, '>', 8.457788467407227, '<=', 0.45192472636699677, '>', -0.7284570932388306, '>', 3.306133508682251, '<=', 0.23589569330215454, '>', 3.5442665815353394, '>', 8.710090160369873]
indices: 428
SORTED:
[2955, 3165, 3166, 3168, 3169, 3298, 3299, 3300, 3301, 3363, 3364, 3365, 3366, 3367, 3368, 3689, 4167, 4503, 4639, 4640, 4703, 4704, 4705, 4707, 4708, 4933, 4934, 4935, 4936, 4937, 4938, 4939, 4940, 4941, 4942, 4943, 4944, 4945, 4946, 4954, 4955, 4956, 4957, 4958, 4959, 4960, 4961, 5007, 5008, 5009, 5010, 5011, 5012, 5013, 5014, 5015, 5016, 5017, 5018, 5019, 5020, 5021, 5022, 5023, 5024, 5025, 5026, 5055, 5056, 5059, 5060, 5064, 5065, 5066, 5067, 5068, 5069, 5070, 5071, 5072, 5073, 5074, 5075, 5076, 5077, 5078, 5079, 5080, 5081, 5082, 5083, 5084, 5085, 5086, 5087, 5088, 5089, 5090, 5091, 5092, 5124, 5125, 5126, 5127, 5128, 5129, 5130, 5134, 5135, 5136, 5141, 5142, 5143, 5144, 5145, 5146, 5147, 5148, 5149, 5150, 5151, 5152, 5153, 

In [ ]:
print("HE RULE:")
###########
rule_neurons_list_he0 = []
rule_neurons = (layer_10_he_rule_neurons0.array[0]).split(",")
for indx in range(0, len(rule_neurons)):
    rule_neurons[indx] = (rule_neurons[indx]).strip()
    rule_neurons[indx] = (rule_neurons[indx]).replace("[", "")
    rule_neurons[indx] = (rule_neurons[indx]).replace("]","")
    rule_neurons_list_he0.append(int(rule_neurons[indx]))

print(rule_neurons_list_he0)

rule_sig_list_he0 = []
rule_sig = (layer_10_he_rule_signature0.array[0]).split(",")
for indx in range(0, len(rule_sig)):
    rule_sig[indx] = (rule_sig[indx]).strip()
    rule_sig[indx] = (rule_sig[indx]).replace("[", "")
    rule_sig[indx] = (rule_sig[indx]).replace("]","")
    if (indx % 2 == 0):
      rule_sig[indx] = (rule_sig[indx]).replace("'", "")
      rule_sig_list_he0.append(rule_sig[indx])
    else:
      rule_sig_list_he0.append(float(rule_sig[indx]))

print(rule_sig_list_he0)

fngprnt = fingerprint_10[0]
indices = get_suffix_cluster(rule_neurons_list_he0, rule_sig_list_he0, fngprnt,VAL=True)
print("indices:", len(indices))

print("SORTED:")
sorted_indices = sorted(indices)
print(sorted_indices)
consecutive_min= [sorted_indices[0]]
consecutive_m = 0
consecutive_max = []
consecutive_l = 0
consecutive_len = []
for indx in range(1, len(sorted_indices)):
  if (sorted_indices[indx] - sorted_indices[indx-1] == 1):
    consecutive_m = sorted_indices[indx]
    consecutive_l = consecutive_l + 1
  else:
    consecutive_max.append(sorted_indices[indx-1])
    consecutive_min.append(sorted_indices[indx])
    consecutive_len.append(consecutive_l)
    consecutive_l = 0
    consecutive_m = 0

print("MIN:", consecutive_min)
print("MAX:", consecutive_max)

max_indx = consecutive_len.index(np.max(consecutive_len))
print("LEN:",np.max(consecutive_len), max_indx)



x_train_he0 = []
fngprnt_he0 = []
inp_he_ex0 = []
finger_he_ex0 = []

for indx in range(consecutive_min[max_indx], consecutive_max[max_indx]):
    if (indx == consecutive_min[max_indx]):
      inp_he_ex0 = x_train_flat[indx]
      finger_he_ex0 = fingerprint_10[0][indx]

    x_train_he0.append(x_train_flat[indx])
    fngprnt_he0.append(fingerprint_10[0][indx])

x_train_he0 = np.array(x_train_he0)
fngprnt_he0 = np.array(fngprnt_he0)

print(np.shape(x_train_he0))
x_train_minhe0 = np.zeros(length)
x_train_maxhe0 = np.zeros(length)

for indx in range(0,length):
  x_train_minhe0[indx] = np.min(x_train_he0[:,indx])
  x_train_maxhe0[indx] = np.max(x_train_he0[:,indx])

print(x_train_minhe0)
print(x_train_maxhe0)

print(np.shape(fngprnt_he0))
fngprnt_min_he0 = np.zeros(len(fngprnt_he0[0]))
fngprnt_max_he0 = np.zeros(len(fngprnt_he0[0]))

for indx in range(0,len(fngprnt_he0[0])):
  fngprnt_min_he0[indx] = np.min(fngprnt_he0[:,indx])
  fngprnt_max_he0[indx] = np.max(fngprnt_he0[:,indx])

print(fngprnt_min_he0)
print(fngprnt_max_he0)


print("INP:", inp_he_ex0)
print("FINGER:", finger_he_ex0)

HE RULE:
[2, 7, 4, 7, 1, 1, 5]
['<=', 5.0200934410095215, '<=', 5.228757619857788, '>', 0.7042066156864166, '<=', 3.8628121614456177, '<=', -1.4256365299224854, '<=', -1.727737545967102, '>', -3.598828911781311]
indices: 236
SORTED:
[2598, 2602, 2603, 2605, 2606, 2607, 2610, 2671, 2672, 2673, 2674, 2675, 2676, 2677, 2678, 2742, 2743, 2744, 2745, 2746, 2747, 2748, 2814, 2815, 2816, 2817, 2818, 2882, 2887, 2888, 2889, 2950, 2951, 2952, 2953, 2954, 3021, 3090, 3091, 3092, 3093, 3094, 3095, 3096, 3159, 3160, 3161, 3162, 3163, 3164, 3226, 3227, 3228, 3229, 3230, 3231, 3232, 3233, 3293, 3294, 3295, 3296, 3297, 3298, 3299, 3300, 3301, 3358, 3359, 3360, 3361, 3362, 3366, 3370, 3432, 3433, 3434, 3435, 3436, 3437, 3438, 3439, 3440, 3498, 3499, 3500, 3501, 3502, 3503, 3504, 3505, 3506, 3507, 3508, 3573, 3574, 3575, 3576, 3578, 3638, 3639, 3640, 3641, 3642, 3643, 3644, 3678, 3679, 3680, 3681, 3682, 3683, 3684, 3685, 3686, 3749, 3750, 3751, 3753, 3754, 3820, 3821, 3888, 3889, 3890, 3891, 3892, 3893

**PROVE RULES USING MARABOU**

**MARABOU SETUP:**

IMPORTANT: INSTALL AND BUILD MARABOU from https://github.com/NeuralNetworkVerification/Marabou.

The following code assumes /content/drive/MyDrive/Marabou_bld to be the Marabou build directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.chdir('/content/drive/MyDrive/Marabou_bld')
!pwd
!ls -lt
path = os.environ['PATH']
print(path)
#os.environ['PATH'] = '/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/Marabou/build/Marabou:/content/Marabou/build/Marabou/build:/content/Marabou/build/Marabou/build/bin:/content/Marabou/build/Marabou:/content/Marabou/build/Marabou/build:/content/Marabou/build/Marabou/build/bin'
print(os.environ['PATH'])
os.environ['PATH'] = path + ':/content/drive/MyDrive/Marabou_bld:/content/drive/MyDrive/Marabou_bld/build:/content/drive/MyDrive/Marabou_bld/build/bin'
print(os.environ['PATH'])

Mounted at /content/drive


In [ ]:
! pip install onnx
! pip install onnx-tf

! pip install pybind11
! pip install cmake

import torch


import onnx
!pip install onnxruntime
import onnxruntime

#import keras

In [ ]:
onnx_model = onnx.load("/content/ProphecyPlus/dataset_models/kj_tiny_taxinet/KJ_TinyTaxiNet.onnx")
onnx.checker.check_model(onnx_model)

print(onnx.helper.printable_graph(onnx_model.graph))

input_names = [input.name for input in onnx_model.graph.input]
print("Input layer names:", input_names)

onnx_outputs = [i.name for i in onnx_model.graph.output]
print("output names:", onnx_outputs)

graph tf2onnx (
  %X_place:0[FLOAT, unk__11x8x16x1]
) optional inputs with matching initializers (
  %dense_2/kernel:0[FLOAT, 8x2]
  %dense_2/bias:0[FLOAT, 2]
  %dense_1/kernel:0[FLOAT, 8x8]
  %dense_1/bias:0[FLOAT, 8]
  %dense/kernel:0[FLOAT, 16x8]
  %dense/bias:0[FLOAT, 8]
  %conv2d/kernel:0[FLOAT, 8x1x8x8]
  %conv2d/bias:0[FLOAT, 8]
  %Reshape/shape:0[INT32, 2]
) {
  %conv2d/Conv2D__6:0 = Transpose[perm = [0, 3, 1, 2]](%X_place:0)
  %conv2d/Conv2D:0 = Conv[dilations = [1, 1], kernel_shape = [8, 8], pads = [0, 0, 0, 0], strides = [8, 8]](%conv2d/Conv2D__6:0, %conv2d/kernel:0)
  %conv2d/Conv2D__8:0 = Transpose[perm = [0, 2, 3, 1]](%conv2d/Conv2D:0)
  %conv2d/BiasAdd:0 = Add(%conv2d/Conv2D__8:0, %conv2d/bias:0)
  %relu:0 = Relu(%conv2d/BiasAdd:0)
  %Reshape__10:0 = Cast[to = 7](%Reshape/shape:0)
  %Reshape:0 = Reshape(%relu:0, %Reshape__10:0)
  %dense/MatMul:0 = MatMul(%Reshape:0, %dense/kernel:0)
  %dense/BiasAdd:0 = Add(%dense/MatMul:0, %dense/bias:0)
  %relu_1:0 = Relu(%dense/BiasAd

**PROVE RULE FOR SAT OF SAFETY PROPERTY |cte| <= 4.3**

In [ ]:
import os
import numpy as np
from maraboupy import Marabou
from maraboupy.MarabouCore import *
from maraboupy.MarabouPythonic import *

options = Marabou.createOptions(verbosity = 1, numWorkers=1, numBlasThreads=1,snc=True)

filename = "/content/ProphecyPlus/dataset_models/kj_tiny_taxinet/KJ_TinyTaxiNet.onnx"
network_a = Marabou.read_onnx(filename)


print("INPUT VARS")
invars = network_a.inputVars[0][0].flatten()



for indx in range(0,len(invars)):
    i = invars[indx]
    v = Var(i)
    network_a.setLowerBound(i,x_train_minCTE[i])
    network_a.setUpperBound(i,x_train_maxCTE[i])
    #network_a.setLowerBound(i,inp_ex[indx])
    #network_a.setUpperBound(i,inp_ex[indx])


print("LAYER VARS MAP")
print(network_a.layerNameToVariables)

dense_10_neurons = network_a.layerNameToVariables["dense_1/BiasAdd:0"][0]
print(np.shape(dense_10_neurons))
print(len(dense_10_neurons))





for indx in range(0, len(dense_10_neurons)):
    neuron_indx = dense_10_neurons[indx] - dense_10_neurons[0]
    network_a.setLowerBound(dense_10_neurons[indx], fngprnt_min_cte[neuron_indx])
    network_a.setUpperBound(dense_10_neurons[indx], fngprnt_max_cte[neuron_indx])
    #network_a.setLowerBound(dense_10_neurons[indx], finger_ex[neuron_indx] - 1.0)
    #network_a.setUpperBound(dense_10_neurons[indx], finger_ex[neuron_indx] + 1.0)

    if (neuron_indx in rule_neurons_list_cte):
      v = Var(dense_10_neurons[indx])

      for indx in range(0, len(rule_neurons_list_cte)):
        if (rule_neurons_list_cte[indx] == neuron_indx):
          index = indx
          sig_indx = (2*index) + 1
          print("N:", neuron_indx, index)
          print("OP:", rule_sig_list_cte[sig_indx-1])
          print("VAL:", rule_sig_list_cte[sig_indx])
          if (rule_sig_list_cte[sig_indx-1] == '<='):
            network_a.addConstraint(float(rule_sig_list_cte[sig_indx]) >= v)
          else:
            network_a.addConstraint(v >= float(rule_sig_list_cte[sig_indx]))



print("OUTPUT VARS")
outvars = network_a.outputVars[0].flatten()
print(outvars)
label_var = Var(outvars[0])

# |cte| >= 4.3 should be UNSAT
# CONSTRAINT 1
network_a.addConstraint(label_var >= 4.3)
# CONSTRAINT 2
#network_a.addConstraint(-4.3 >= label_var)



INPUT VARS
LAYER VARS MAP
{'X_place:0': array([[[[  0],
         [  1],
         [  2],
         [  3],
         [  4],
         [  5],
         [  6],
         [  7],
         [  8],
         [  9],
         [ 10],
         [ 11],
         [ 12],
         [ 13],
         [ 14],
         [ 15]],

        [[ 16],
         [ 17],
         [ 18],
         [ 19],
         [ 20],
         [ 21],
         [ 22],
         [ 23],
         [ 24],
         [ 25],
         [ 26],
         [ 27],
         [ 28],
         [ 29],
         [ 30],
         [ 31]],

        [[ 32],
         [ 33],
         [ 34],
         [ 35],
         [ 36],
         [ 37],
         [ 38],
         [ 39],
         [ 40],
         [ 41],
         [ 42],
         [ 43],
         [ 44],
         [ 45],
         [ 46],
         [ 47]],

        [[ 48],
         [ 49],
         [ 50],
         [ 51],
         [ 52],
         [ 53],
         [ 54],
         [ 55],
         [ 56],
         [ 57],
         [ 58],
         [

In [ ]:
#print(network_a.getInputQuery())
sat_unsat,vals, stats = network_a.solve(options = options)

print("sat_unsat:", sat_unsat)
print("vals:", vals)
print("stats:", stats)


unsat
sat_unsat: unsat
vals: {}
stats: <maraboupy.MarabouCore.Statistics object at 0x7bb4aef9e1f0>


**PROVE RULE FOR VIOLATION OF SAFETY PROPERTY |cte| <= 4.3**

In [ ]:
import os
import numpy as np
from maraboupy import Marabou
from maraboupy.MarabouCore import *
from maraboupy.MarabouPythonic import *

options = Marabou.createOptions(verbosity = 1, numWorkers=1, numBlasThreads=1,snc=True)

filename = "/content/ProphecyPlus/dataset_models/kj_tiny_taxinet/KJ_TinyTaxiNet.onnx"
network_a = Marabou.read_onnx(filename)


print("INPUT VARS")
invars = network_a.inputVars[0][0].flatten()



for indx in range(0,len(invars)):
    i = invars[indx]
    v = Var(i)
    network_a.setLowerBound(i,x_train_minCTE0[i])
    network_a.setUpperBound(i,x_train_maxCTE0[i])
    #network_a.setLowerBound(i,inp_ex0[indx])
    #network_a.setUpperBound(i,inp_ex0[indx])


print("LAYER VARS MAP")
print(network_a.layerNameToVariables)

dense_10_neurons = network_a.layerNameToVariables["dense_1/BiasAdd:0"][0]
print(np.shape(dense_10_neurons))
print(len(dense_10_neurons))





for indx in range(0, len(dense_10_neurons)):
    neuron_indx = dense_10_neurons[indx] - dense_10_neurons[0]
    network_a.setLowerBound(dense_10_neurons[indx], fngprnt_min_cte0[neuron_indx])
    network_a.setUpperBound(dense_10_neurons[indx], fngprnt_max_cte0[neuron_indx])
   # network_a.setLowerBound(dense_10_neurons[indx], finger_ex0[neuron_indx] - 1.0)
   # network_a.setUpperBound(dense_10_neurons[indx], finger_ex0[neuron_indx] + 1.0)

    if (neuron_indx in rule_neurons_list_cte0):
      v = Var(dense_10_neurons[indx])

      for indx in range(0, len(rule_neurons_list_cte0)):
        if (rule_neurons_list_cte0[indx] == neuron_indx):
          index = indx
          sig_indx = (2*index) + 1
          print("N:", neuron_indx, index)
          print("OP:", rule_sig_list_cte0[sig_indx-1])
          print("VAL:", rule_sig_list_cte0[sig_indx])
          if (rule_sig_list_cte0[sig_indx-1] == '<='):
            network_a.addConstraint(float(rule_sig_list_cte0[sig_indx]) >= v)
          else:
            network_a.addConstraint(v >= float(rule_sig_list_cte0[sig_indx]))



print("OUTPUT VARS")
outvars = network_a.outputVars[0].flatten()
print(outvars)
label_var = Var(outvars[0])

# |cte| < 4.3  should be UNSAT

network_a.addConstraint(4.3 >= label_var)
network_a.addConstraint(label_var >= -4.3)



INPUT VARS
LAYER VARS MAP
{'X_place:0': array([[[[  0],
         [  1],
         [  2],
         [  3],
         [  4],
         [  5],
         [  6],
         [  7],
         [  8],
         [  9],
         [ 10],
         [ 11],
         [ 12],
         [ 13],
         [ 14],
         [ 15]],

        [[ 16],
         [ 17],
         [ 18],
         [ 19],
         [ 20],
         [ 21],
         [ 22],
         [ 23],
         [ 24],
         [ 25],
         [ 26],
         [ 27],
         [ 28],
         [ 29],
         [ 30],
         [ 31]],

        [[ 32],
         [ 33],
         [ 34],
         [ 35],
         [ 36],
         [ 37],
         [ 38],
         [ 39],
         [ 40],
         [ 41],
         [ 42],
         [ 43],
         [ 44],
         [ 45],
         [ 46],
         [ 47]],

        [[ 48],
         [ 49],
         [ 50],
         [ 51],
         [ 52],
         [ 53],
         [ 54],
         [ 55],
         [ 56],
         [ 57],
         [ 58],
         [

In [ ]:
sat_unsat,vals, stats = network_a.solve(options = options)

print("sat_unsat:", sat_unsat)
print("vals:", vals)
print("stats:", stats)


sat
input 0 = 0.5147843903186273
input 1 = 0.5255150888480392
input 2 = 0.5238530177696078
input 3 = 0.5336435355392156
input 4 = 0.4821920955882354
input 5 = 0.4499789368872549
input 6 = 0.444592524509804
input 7 = 0.4424343055754672
input 8 = 0.4345492493872549
input 9 = 0.49626799938725497
input 10 = 0.5520890778186274
input 11 = 0.5668907015931373
input 12 = 0.563558900122549
input 13 = 0.5467447916666667
input 14 = 0.5383597579656864
input 15 = 0.6567523192466
input 16 = 0.5230373008578431
input 17 = 0.5468979779411766
input 18 = 0.5260646446078432
input 19 = 0.5069469975490197
input 20 = 0.4908624387254903
input 21 = 0.4652581232179744
input 22 = 0.4384114583333334
input 23 = 0.43405330882352955
input 24 = 0.4157111672794117
input 25 = 0.3700769761029412
input 26 = 0.4428768382352942
input 27 = 0.44875344669117645
input 28 = 0.5243489583333334
input 29 = 0.6140548406862746
input 30 = 0.6264016544117648
input 31 = 0.6244944852941178
input 32 = 0.5272901348039216
input 33 = 0.52708

**PROVE RULE FOR SAT OF SAFETY PROPERTY |he| <= 8.03**

In [ ]:
import os
import numpy as np
from maraboupy import Marabou
from maraboupy.MarabouCore import *
from maraboupy.MarabouPythonic import *

options = Marabou.createOptions(verbosity = 1, numWorkers=1, numBlasThreads=1,snc=True)

filename = "/content/ProphecyPlus/dataset_models/kj_tiny_taxinet/KJ_TinyTaxiNet.onnx"
network_a = Marabou.read_onnx(filename)


print("INPUT VARS")
invars = network_a.inputVars[0][0].flatten()



for indx in range(0,len(invars)):
    i = invars[indx]
    v = Var(i)
    network_a.setLowerBound(i,x_train_minhe[i])
    network_a.setUpperBound(i,x_train_maxhe[i])
    #network_a.setLowerBound(i,inp_he_ex[indx])
    #network_a.setUpperBound(i,inp_he_ex[indx])


print("LAYER VARS MAP")
print(network_a.layerNameToVariables)

dense_10_neurons = network_a.layerNameToVariables["dense_1/BiasAdd:0"][0]
print(np.shape(dense_10_neurons))
print(len(dense_10_neurons))





for indx in range(0, len(dense_10_neurons)):
    neuron_indx = dense_10_neurons[indx] - dense_10_neurons[0]
    network_a.setLowerBound(dense_10_neurons[indx], fngprnt_min_he[neuron_indx])
    network_a.setUpperBound(dense_10_neurons[indx], fngprnt_max_he[neuron_indx])
    #network_a.setLowerBound(dense_10_neurons[indx], finger_he_ex[neuron_indx] - 1.0)
    #network_a.setUpperBound(dense_10_neurons[indx], finger_he_ex[neuron_indx] + 1.0)

    if (neuron_indx in rule_neurons_list_he):
      v = Var(dense_10_neurons[indx])

      for indx in range(0, len(rule_neurons_list_he)):
        if (rule_neurons_list_he[indx] == neuron_indx):
          index = indx
          sig_indx = (2*index) + 1
          print("N:", neuron_indx, index)
          print("OP:", rule_sig_list_he[sig_indx-1])
          print("VAL:", rule_sig_list_he[sig_indx])
          if (rule_sig_list_he[sig_indx-1] == '<='):
            network_a.addConstraint(float(rule_sig_list_he[sig_indx]) >= v)
          else:
            network_a.addConstraint(v >= float(rule_sig_list_he[sig_indx]))



print("OUTPUT VARS")
outvars = network_a.outputVars[0].flatten()
print(outvars)
label_var = Var(outvars[1])

# |he| >= 8.03 should be UNSAT
# CONSTRAINT 1
#network_a.addConstraint(label_var >= 8.03)
# CONSTRAINT 2
network_a.addConstraint(-8.0 >= label_var)



INPUT VARS
LAYER VARS MAP
{'X_place:0': array([[[[  0],
         [  1],
         [  2],
         [  3],
         [  4],
         [  5],
         [  6],
         [  7],
         [  8],
         [  9],
         [ 10],
         [ 11],
         [ 12],
         [ 13],
         [ 14],
         [ 15]],

        [[ 16],
         [ 17],
         [ 18],
         [ 19],
         [ 20],
         [ 21],
         [ 22],
         [ 23],
         [ 24],
         [ 25],
         [ 26],
         [ 27],
         [ 28],
         [ 29],
         [ 30],
         [ 31]],

        [[ 32],
         [ 33],
         [ 34],
         [ 35],
         [ 36],
         [ 37],
         [ 38],
         [ 39],
         [ 40],
         [ 41],
         [ 42],
         [ 43],
         [ 44],
         [ 45],
         [ 46],
         [ 47]],

        [[ 48],
         [ 49],
         [ 50],
         [ 51],
         [ 52],
         [ 53],
         [ 54],
         [ 55],
         [ 56],
         [ 57],
         [ 58],
         [

In [ ]:
#print(network_a.getInputQuery())
sat_unsat,vals, stats = network_a.solve(options = options)

print("sat_unsat:", sat_unsat)
print("vals:", vals)
print("stats:", stats)


sat
input 0 = 0.5766505821078431
input 1 = 0.5717294730392156
input 2 = 0.5354568780637257
input 3 = 0.5357211243872549
input 4 = 0.5533413756127451
input 5 = 0.5349858302696079
input 6 = 0.4996419270833333
input 7 = 0.4895349274293377
input 8 = 0.534704350490196
input 9 = 0.6067497702205883
input 10 = 0.5388576133578432
input 11 = 0.5346909466911766
input 12 = 0.5418485753676471
input 13 = 0.5326803768382353
input 14 = 0.5296894148284315
input 15 = 0.526849724264706
input 16 = 0.44483379289215685
input 17 = 0.4352309283088235
input 18 = 0.3614066329656863
input 19 = 0.5987113204656863
input 20 = 0.5507678462009804
input 21 = 0.4415536917892158
input 22 = 0.4626321231617647
input 23 = 0.47709290747549027
input 24 = 0.49253408394607845
input 25 = 0.4963957291806721
input 26 = 0.6368987438725491
input 27 = 0.6988568474264707
input 28 = 0.5648380055147059
input 29 = 0.5348862591911765
input 30 = 0.5324352787990196
input 31 = 0.5302293964460785
input 32 = 0.5466509650735294
input 33 = 0.45

**PROVE RULE FOR VIOLATION OF SAFETY PROPERTY |he| <= 8.03**

In [ ]:
import os
import numpy as np
from maraboupy import Marabou
from maraboupy.MarabouCore import *
from maraboupy.MarabouPythonic import *

options = Marabou.createOptions(verbosity = 1, numWorkers=1, numBlasThreads=1,snc=True)

filename = "/content/ProphecyPlus/dataset_models/kj_tiny_taxinet/KJ_TinyTaxiNet.onnx"
network_a = Marabou.read_onnx(filename)


print("INPUT VARS")
invars = network_a.inputVars[0][0].flatten()



for indx in range(0,len(invars)):
    i = invars[indx]
    v = Var(i)
    network_a.setLowerBound(i,x_train_minhe0[i])
    network_a.setUpperBound(i,x_train_maxhe0[i])
    #network_a.setLowerBound(i,inp_he_ex0[indx])
    #network_a.setUpperBound(i,inp_he_ex0[indx])


print("LAYER VARS MAP")
print(network_a.layerNameToVariables)

dense_10_neurons = network_a.layerNameToVariables["dense_1/BiasAdd:0"][0]
print(np.shape(dense_10_neurons))
print(len(dense_10_neurons))





for indx in range(0, len(dense_10_neurons)):
    neuron_indx = dense_10_neurons[indx] - dense_10_neurons[0]
    network_a.setLowerBound(dense_10_neurons[indx], fngprnt_min_he0[neuron_indx])
    network_a.setUpperBound(dense_10_neurons[indx], fngprnt_max_he0[neuron_indx])
    #network_a.setLowerBound(dense_10_neurons[indx], finger_he_ex0[neuron_indx] - 1.0)
    #network_a.setUpperBound(dense_10_neurons[indx], finger_he_ex0[neuron_indx] + 1.0)

    if (neuron_indx in rule_neurons_list_he0):
      v = Var(dense_10_neurons[indx])

      for indx in range(0, len(rule_neurons_list_he0)):
        if (rule_neurons_list_he0[indx] == neuron_indx):
          index = indx
          sig_indx = (2*index) + 1
          print("N:", neuron_indx, index)
          print("OP:", rule_sig_list_he0[sig_indx-1])
          print("VAL:", rule_sig_list_he0[sig_indx])
          if (rule_sig_list_he0[sig_indx-1] == '<='):
            network_a.addConstraint(float(rule_sig_list_he0[sig_indx]) >= v)
          else:
            network_a.addConstraint(v >= float(rule_sig_list_he0[sig_indx]))



print("OUTPUT VARS")
outvars = network_a.outputVars[0].flatten()
print(outvars)
label_var = Var(outvars[1])

# |he| < 8.03 should be UNSAT
network_a.addConstraint(8.03 >= label_var)
network_a.addConstraint(label_var >= -8.03)


INPUT VARS
LAYER VARS MAP
{'X_place:0': array([[[[  0],
         [  1],
         [  2],
         [  3],
         [  4],
         [  5],
         [  6],
         [  7],
         [  8],
         [  9],
         [ 10],
         [ 11],
         [ 12],
         [ 13],
         [ 14],
         [ 15]],

        [[ 16],
         [ 17],
         [ 18],
         [ 19],
         [ 20],
         [ 21],
         [ 22],
         [ 23],
         [ 24],
         [ 25],
         [ 26],
         [ 27],
         [ 28],
         [ 29],
         [ 30],
         [ 31]],

        [[ 32],
         [ 33],
         [ 34],
         [ 35],
         [ 36],
         [ 37],
         [ 38],
         [ 39],
         [ 40],
         [ 41],
         [ 42],
         [ 43],
         [ 44],
         [ 45],
         [ 46],
         [ 47]],

        [[ 48],
         [ 49],
         [ 50],
         [ 51],
         [ 52],
         [ 53],
         [ 54],
         [ 55],
         [ 56],
         [ 57],
         [ 58],
         [

In [ ]:
#print(network_a.getInputQuery())
sat_unsat,vals, stats = network_a.solve(options = options)

print("sat_unsat:", sat_unsat)
print("vals:", vals)
print("stats:", stats)


sat
input 0 = 0.5849207261029412
input 1 = 0.6312442555147059
input 2 = 0.6361883425245098
input 3 = 0.6180319393382353
input 4 = 0.6040268841911764
input 5 = 0.5852079503676471
input 6 = 0.5290383731617647
input 7 = 0.6033643535539216
input 8 = 0.5032915900735295
input 9 = 0.5796875
input 10 = 0.6214403339460784
input 11 = 0.6454829197303922
input 12 = 0.6614506740196078
input 13 = 0.584734987745098
input 14 = 0.5844305300245098
input 15 = 0.5832643995098039
input 16 = 0.6589824601715686
input 17 = 0.6055089613970588
input 18 = 0.5273648131127451
input 19 = 0.5133750765931373
input 20 = 0.49006778492647063
input 21 = 0.45060700061274517
input 22 = 0.5366182675823071
input 23 = 0.6370404411764706
input 24 = 0.4127814797794117
input 25 = 0.4588541666666667
input 26 = 0.48973651960784315
input 27 = 0.5083639705882353
input 28 = 0.5442606927367828
input 29 = 0.7048330269607843
input 30 = 0.676629518995098
input 31 = 0.6669730392156862
input 32 = 0.5002814797794118
input 33 = 0.51188151041